## Clasificador de Noticias Argentinas 

In [2]:
import pandas as pd

df = pd.read_excel('../../res/tp1/Noticias_argentinas.xlsx', sheet_name='aa_bayes')

In [3]:
# Nos quedamos con los atributos de interes

df = df[["titular", "categoria"]]

# removemos las categorias que no necesitamos ('Destacadas', 'Noticias Destacadas')


df.head()

,titular,categoria
0,Trabajadores del Buenos Aires Design cortan la...,Nacional
1,La boda del gobernador Gerardo Morales: tapas ...,Nacional
2,Cumbre del G20: qué calles estarán cortadas y ...,Nacional
3,Una fractura que confirma la candidatura de Cr...,Nacional
4,Infierno grande: ola de divorcios en un pueblo...,Nacional


In [4]:
df = df.loc[df['categoria'].isin(("Nacional", "Economia", "Internacional", "Deportes", "Salud", "Ciencia y Tecnologia", "Entretenimiento"))]
df.head()
categories = df['categoria'].unique()
print(categories)


['Nacional' 'Deportes' 'Salud' 'Ciencia y Tecnologia' 'Entretenimiento'
 'Economia' 'Internacional']


In [5]:
len(df)

26961

In [6]:
df['categoria'].value_counts()

Nacional                3860
Ciencia y Tecnologia    3856
Deportes                3855
Entretenimiento         3850
Internacional           3850
Economia                3850
Salud                   3840
Name: categoria, dtype: int64

In [7]:
from sklearn.model_selection import train_test_split

# Separar los datos en datos de entrenamiento y testeo
train, test = train_test_split(df, test_size=0.2)
train.head()
test.head()

,titular,categoria
21848,Un banco deberá pagarle $40 mil a un cliente p...,Nacional
29870,River-Boca: la final de la Copa Libertadores p...,Deportes
20849,Testeos rápidos gratuitos y prevención de VIH/...,Salud
6811,Dramático llamado de la ONU tras otro récord d...,Internacional
323,"Mauricio Macri, sobre los atentados: 'Vamos a ...",Nacional


In [8]:
textos = list(df['titular'])

In [9]:
# Crear diccionario de categorias con diccionarios vacios para las palabras de los titulares
categories_word_appearances = { i : {} for i in categories }

In [10]:
print(categories_word_appearances)

{'Nacional': {}, 'Deportes': {}, 'Salud': {}, 'Ciencia y Tecnologia': {}, 'Entretenimiento': {}, 'Economia': {}, 'Internacional': {}}


In [11]:
import re

def split_and_sanitize(title):
    words = title.split()  # se separan las palabras según espacios (quita todos los espacios)
    words = list(map(lambda x: re.sub("[^\w\s]", '', x), words))  # reemplazo de símbolos por ningún caracter
    return words

for i in range(len(train)):
    row = train.iloc[i]
    category = row['categoria']
    title = row['titular']
    words = split_and_sanitize(title)
    
    for word in words:
            if word in categories_word_appearances[category]:
                categories_word_appearances[category][word] += 1
            else:
                categories_word_appearances[category][word] = 1
                
    
    
    

In [12]:
print(categories_word_appearances['Entretenimiento']['Pampita'])

70


In [13]:
# Calcular probabilidades de las clases P(category) para el conjunto de entrenamiento

category_relative_freq = train['categoria'].value_counts() / len(train)  # P(category)
print(category_relative_freq)

# Calcular las probabilidades condicionales P(word | category)
word_relative_frequencies = { i : {} for i in categories }
zero_probabilities = { i : 0 for i in categories } # Laplace correction in case a word does not appear in a category
for category in categories:
    word_appearances_sum = sum(categories_word_appearances[category].values())
    laplace_denominator = word_appearances_sum + len(categories_word_appearances[category].keys())
    zero_probabilities[category] = 1 / laplace_denominator
    for word, word_count in categories_word_appearances[category].items():
        word_relative_frequencies[category][word] = (word_count + 1) / laplace_denominator

print(word_relative_frequencies['Entretenimiento']['Pampita'])
# Objetivo final:
#   P (category | [word])
# = P ([word] | category) * P (category)      ##  / P ([word])
# = prod(P (word | category)) * P (category)

Deportes                0.144473
Ciencia y Tecnologia    0.144010
Internacional           0.143175
Nacional                0.142711
Entretenimiento         0.142619
Economia                0.141784
Salud                   0.141228
Name: categoria, dtype: float64
0.0015191390119177525


In [14]:
def classify(title):
    words = split_and_sanitize(title)
    vs = {}
    for category in categories:
        v = category_relative_freq.loc[category]
        for word in words:
            if word in word_relative_frequencies[category]:
                v *= word_relative_frequencies[category][word]
            else:
                v *= zero_probabilities[category]
        vs[category] = v
    mx = (None, 0)
    for category, v in vs.items():
        if v > mx[1]:
            mx = (category, v)
    return mx[0]

noticia = 'Pampita fue a la mesa de Mirtha'
print(f"La noticia '{noticia}' pertenece a {classify(noticia)}")
noticia = 'Messi metió un gol'
print(f"La noticia '{noticia}' pertenece a {classify(noticia)}")
noticia = 'Apple lanzó un nuevo iPhone'
print(f"La noticia '{noticia}' pertenece a {classify(noticia)}")

La noticia 'Pampita fue a la mesa de Mirtha' pertenece a Entretenimiento
La noticia 'Messi metió un gol' pertenece a Deportes
La noticia 'Apple lanzó un nuevo iPhone' pertenece a Ciencia y Tecnologia


In [27]:
confusion_matrix = { i : {} for i in categories }
columns = []
for key in confusion_matrix.keys():
    columns.append(key[0:3])
    confusion_matrix[key] = { i : 0 for i in categories } # {'Deportes': {'Deportes': 700, 'Entretenimiento': 3, ...}, ...}

for i in range(len(test)):
    row = test.iloc[i]
    category = row['categoria']
    title = row['titular']
    result = classify(title)
    confusion_matrix[category][result] += 1

print(confusion_matrix)
confusion_df = { key : list(confusion_matrix[key].values()) for key in confusion_matrix.keys() }
print('')
print(confusion_df)
confusion_df = pd.DataFrame.from_dict(confusion_df, orient='index', columns=columns)
print('')
print(confusion_df)

{'Nacional': {'Nacional': 720, 'Deportes': 13, 'Salud': 10, 'Ciencia y Tecnologia': 0, 'Entretenimiento': 10, 'Economia': 19, 'Internacional': 10}, 'Deportes': {'Nacional': 8, 'Deportes': 709, 'Salud': 2, 'Ciencia y Tecnologia': 2, 'Entretenimiento': 13, 'Economia': 0, 'Internacional': 5}, 'Salud': {'Nacional': 1, 'Deportes': 2, 'Salud': 781, 'Ciencia y Tecnologia': 3, 'Entretenimiento': 0, 'Economia': 3, 'Internacional': 4}, 'Ciencia y Tecnologia': {'Nacional': 0, 'Deportes': 1, 'Salud': 1, 'Ciencia y Tecnologia': 733, 'Entretenimiento': 4, 'Economia': 6, 'Internacional': 5}, 'Entretenimiento': {'Nacional': 5, 'Deportes': 4, 'Salud': 5, 'Ciencia y Tecnologia': 4, 'Entretenimiento': 745, 'Economia': 6, 'Internacional': 5}, 'Economia': {'Nacional': 3, 'Deportes': 0, 'Salud': 7, 'Ciencia y Tecnologia': 16, 'Entretenimiento': 2, 'Economia': 754, 'Internacional': 10}, 'Internacional': {'Nacional': 32, 'Deportes': 6, 'Salud': 21, 'Ciencia y Tecnologia': 2, 'Entretenimiento': 13, 'Economia':